# This notebook shows how to extract a model from a Latex document and simulate the mode. 

## Why specify a model in Latex? 

Sometime the **implementation** of a model in software don't match the **specification** of the model in 
the text in which the model is presented. It can be a challenge to make sure that the specification is 
updated in order to reflect changes made in the implementation. 

By extracting the model from a Latex script which describes and specify the model a one can always be sure that simulations reflect the model as described in the paper. 

Also the author is forced to make a complete specification of the model, else it won't run. 

## The Economic Credit Loss model 
This jupyter notebook is inspired by IMF working paper (WP/20/111) The Expected Credit Loss Modeling from a Top-Down Stress Testing Perspective by Marco Gross, Dimitrios Laliotis, Mindaugas Leika, Pavel Lukyantsau. The working paper and the associated material is located https://www.imf.org/en/Publications/WP/Issues/2020/07/03/Expected-Credit-Loss-Modeling-from-a-Top-Down-Stress-Testing-Perspective-49545

from the abstract of the paper: 
> The objective of this paper is to present an integrated tool suite for IFRS 9- and CECL compatible
estimation in top-down solvency stress tests. The tool suite serves as an
illustration for institutions wishing to include accounting-based approaches for credit risk
modeling in top-down stress tests.

This is a jupyter notebook build with the purpose of illustrating the conversion of a model from Latex to ModelFlow. The purpose is testing so take very much care.  

## Import libraries 

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from IPython.core.display import HTML,Markdown


from modelclass import model
import modeljupytermagic

# some useful stuf
model.widescreen()
pd.set_option('display.max_rows', None, 'display.max_columns', 10, 'precision', 2)
sortdf = lambda df: df[sorted([c for c in df.columns])]

OptionError: 'Pattern matched multiple keys'

## Write a latex script
The model consists of the equations and the lists 

The jupyter magic command **%%latexflow** will extract the model. then it will  transform the equations to **ModelFlow** equations and finaly it will create a modelflow **model** instance. 

The **model** instance will be able to solve the model. 

In [ ]:
%%latexflow ecl


### Loans can be in 3 stages
Loans can be in 3 stages, s1,s2 and s3. 
New loans will be generated and loans will mature. 


Two lists of stages are defined: 
    
List $stage=\{s1, s2,s3\}$

List $stage\_from=\{s1, s2,s3\}$

### A share of the loans in each stage wil transition to the same or another stage in the next time frame:  

\begin{equation}
\label{eq:loanfromto}
loan\_transition\_from\_to^{stage\_from,stage}_{t} = loan^{stage\_from}_{t-1} \times TR^{stage\_from,stage}_{t} 
\end{equation}

\begin{equation}
\label{eq:transition}
loan\_transition\_to^{stage}_{t}  = \underbrace{\sum_{stage\_from}(loan\_transition\_from\_to^{stage\_from,stage}_{t})}_{Transition}
\end{equation}

### A share of the loans in each stage will mature another share will have to be written off

\begin{equation}
\label{eq:maturing}
loan\_maturing^{stage}_{t}  = M^{stage}_{t} \times loan^{stage}_{t-1}
\end{equation}

\begin{equation}
\label{eq:writeoff}
loan\_writeoff^{stage}_{t}  = WRO^{stage}_{t} \times loan^{stage}_{t-1}
\end{equation}

### So loans in a stage will reflect the inflow and outflow

\begin{equation}
\label{eq:E}
loan^{stage}_{t}  = \underbrace{loan\_transition\_to^{stage}_{t} }_{Transition}
-\underbrace{loan\_maturing^{stage}_{t}}_{Maturing}
-\underbrace{loan\_writeoff^{stage}_{t}}_{Writeoff}
+\underbrace{loan\_new^{stage}_{t}}_{New loans}
\end{equation}


\begin{equation}
\label{eq:new}
loan\_new^{stage}_{t}  = new^{stage}_{t} \times loan^{stage}_{t-1}
\end{equation}
 
\begin{equation}
\label{eq:g}
loan\_total_{t}  =  \sum_{stage}(loan^{stage}_{t})
\end{equation}


### New loans are only generated in stage 1.

\begin{equation}
\label{eq:E2}
new\_s1_{t} = \frac{loan\_growth_{t} \times loan\_total_{t-1} + 
                \sum_{stage}((M^{stage}_{t}+WRO^{stage}_{t})\times loan^{stage}_{t-1})}{(loan\_s1_{t-1})}
\end{equation}

### Performing Loans 

\begin{equation}
\label{eq:Performing}
loan\_performing_{t} = loan\_s1_{t}+loan\_s2_{t}
\end{equation}

### Cure 

\begin{equation}
\label{eq:cure}
loan\_cure = loan\_transition\_from\_to\_s3\_s1+loan\_transition\_from\_to\_s3\_s2
\end{equation}

### Probability of default (PD)

The point in time PD is the fraction of loans in stage s1 and s2 going into stage s3 
\begin{equation}
\label{eq:PDPIT}
PD\_pit= \frac{loan\_transition\_from\_to\_s1\_s3+loan\_transition\_from\_to\_s2\_s3}{loan\_s1+loan\_s2}
\end{equation}

The Troug The  Cycle PD is a slow mowing average of the Point in time PD. The 


\begin{equation}
\label{eq:PDTTC}
PD\_TTC = logit^{-1}(logit(PD\_TTC(-1)) + alfa \times \Delta{PD\_pit})
\end{equation}

### And we can specify the dynamic of the transition matrix, based on Z score

Let $\Phi$ be the normal cumulative distribution $\frac{1}{\sqrt{2\pi}}
\int_{-\infty}^x e^{
-\frac{t^2}{2}}dt$



\begin{equation}
\label{eq:tr}
TR^{stage\_from,stage}_{t} = \Phi{\left(\frac{bound\_upper^{stage\_from,stage}-\sqrt{\rho}\times Z_{t}}{\sqrt{1-\rho}}\right)}-\Phi{\left(\frac{bound\_lower^{stage\_from,stage}-\sqrt{\rho}\times Z_{t}}{\sqrt{1-\rho}}\right)}
\end{equation}




## The equations in Macro business logic language can be inspected

In [ ]:
print(ecl.equations_original)

### The equations in business logic language can be inspected

In [ ]:
print(ecl.equations)

### The model structure can be inspected 

In [ ]:
ecl.drawmodel(sink='LOAN_TOTAL',HR=0,pdf=0,att=0,size=(12,12))

## load data
The data is copy-pasted from the excel sheet 
### Load data common for baseline and adverse

In [ ]:
%%dataframe T0tr noshow  prefix=TR_  periods=1 melt 
	S1	S2	S3
S1	89%	9%	2%
S2	15%	79%	6%
S3	2%	24%	74%

In [ ]:
%%dataframe  startvalues  show  periods=1 
     pd_pit lgd_pit
        1.6%   20%

In [ ]:
%%dataframe  Loan_t0  show periods=1 melt 
S1	S2	S3 total 
loan 500	180	30 710


In [ ]:
%%dataframe upper_bin prefix=bound_upper_ nshow periods=7 melt 
	S1	S2	S3
S1	10000	-1,34	-2,33
S2	10000	1,08	-1,64
S3	10000	1,88	0,58
 

In [ ]:
%%dataframe lower_bin prefix=bound_lower_ show periods=7 melt 
	S1	S2	S3
S1	-1,34	-2,33	-10000
S2	1,08	-1,64	-10000
S3	1,88	0,58	-10000

In [ ]:
%%dataframe parameters show periods=7  
rho
0.2

### Create the Static dataframe, common to scenarios

In [ ]:
staticdf = pd.concat([T0tr_melted,Loan_t0_melted,startvalues,upper_bin_melted, 
                      lower_bin_melted,parameters],axis=1)
HTML(staticdf.T.style.render().replace('nan',''))

### Load data specific for the scenarios

In [ ]:
%%dataframe  inf_baseline  nshow t periods=7 melt
    m     wro
s1  5%   0
s2  3.8%   0
s3  0    7.5%

In [ ]:
%%dataframe  inf_adverse  nshow t periods=7 melt
    m     wro
s1  3.8%   0
s2  2.5%   0
s3  0    6.3%

In [ ]:
%%dataframe projection_baseline nshow
Z  loan_growth
 0          0.01 
-0,47	    0.01   
-0,42	    0.01
-0,38	    0.01
-0,36	    0.01
-0,34	    0.01
-0,33	    0.01

In [ ]:
%%dataframe projection_adverse nshow
 Z        loan_growth
 0          0.01
  -0,65     -0.01
  -0,84     -0.008
   -0,99    -0.006
    -0,69   -0.004
    -0,39   -0.002
    -0,24   -0.0

### Create a dataframe for baseline and adverse scenario 

In [ ]:
baseupdate    = pd.concat([inf_baseline_melted, projection_baseline],axis=1).pipe(sortdf)
adverseupdate =  pd.concat([inf_adverse_melted, projection_adverse],axis=1).pipe(sortdf)
getnotnul = lambda x: x.loc[:,(x!=0.0).any(axis=0)] 

display(Markdown('## Baseline scenario'))
display(baseupdate.pipe(getnotnul))
display(Markdown('## Adverse scenario'))
display(adverseupdate.pipe(getnotnul))

In [ ]:
baseline = pd.concat([staticdf, inf_baseline_melted, projection_baseline],axis=1)
adverse  = pd.concat([staticdf, inf_adverse_melted, projection_adverse],axis=1)
baseline.index = pd.period_range(start=2021,freq = 'Y',periods=7)
adverse.index  = pd.period_range(start=2021,freq = 'Y',periods=7)

display(baseline.pipe(sortdf).T)

## Run baseline 

In [ ]:
base_result  = ecl(baseline,keep='Baseline')

### Save model and baseline results

In [ ]:
ecl.modeldump('ecl.pcim')

## Run adverse

In [ ]:
adverse_result = ecl(adverse,keep = 'Adverse')

## Inspect Results

In [ ]:
with ecl.set_smpl('2021','2027'):
    ecl.keep_plot('loan_total',showtype='growth',legend=False);

In [ ]:
ecl.keep_plot('tr_*',showtype='level',legend=False);

In [ ]:
ecl.PD_PIT.tracepre(up=4,HR=1,filter=30,pdf=1)